# Day 2

Full text of the task can be found on [AoC website](https://adventofcode.com/2023/day/2).

We arrive at the `Snow Island`, how exciting! We are greated by an Elf who wants to play a game while we walk!

This is how ChatGPT 4 with DALL·E illustrated the description of this tasks.

<img src="./ChatGPT_illustrations/day02.png" width="400" />

## Part One

The game includes the Elf hiding some cubes of different colors in the bag. We then draw sets of cubes from the bag (putting the cubes back after drawing) three times. 

The input consists of game number and the sets of drawn cubes separated by the semicolon. The questions is to determine whether the number of cubes hidden during a given game is compatible with the given set of cubes. 

Example:

```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```

**Q:** Which games would be possible if only 12 red cubes, 13 green cubes, and 14 blue cubes were hidden in the bag?

**A:** 1, 2, and 5.


In [1]:
example_string = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"""

I will split this task, first I want to process the string with set details into a dictionary.

In [2]:
def note_set(ball_set):
    """
    Converts a string of the form 'number color' into a dict of the form {'color': number}
    Example: '3 green, 7 blue, 11 red' -> {'green': 3, 'blue': 7, 'red': 11}
    """
    # make sure ball_set is a string
    ball_set = str(ball_set)
    # strip out of leading and trailing whitespace
    ball_set = ball_set.strip()
    ball_set = ball_set.split(", ")
    ball_set = [ball.split(" ") for ball in ball_set]
    ball_set = {ball[1]: int(ball[0]) for ball in ball_set}
    return ball_set

note_set("3 green, 7 blue, 11 red")

{'green': 3, 'blue': 7, 'red': 11}

Then, given a list of dictionaries I want the highest value for each key.

In [3]:
def take_max_per_key(list_of_dicts):
    """
    Takes a list of dicts and returns a dict with the max value for each key
    """
    # initialize empty dict
    max_dict = {}
    # loop through each dict in list_of_dicts
    for dict_ in list_of_dicts:
        # loop through each key in dict_
        for key in dict_:
            # if key not in max_dict, add it
            if key not in max_dict:
                max_dict[key] = dict_[key]
            # if key in max_dict, compare values and keep the max
            else:
                if dict_[key] > max_dict[key]:
                    max_dict[key] = dict_[key]
    return max_dict

take_max_per_key([{'green': 3, 'blue': 7, 'red': 11}, {'green': 5, 'blue': 2, 'red': 1}])

{'green': 5, 'blue': 7, 'red': 11}

Now, using those two functions I want to process an input string.

In [4]:
def process_input(input_string):
    """
    Takes a string of the form 'Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green'
    and returns a dict of the form {game: {color: number}}
    """
    # create a dictionary of games
    games = {}
    for line in input_string.splitlines():
        if line:
            game, cubes = line.split(':')
            game = int(game.split(" ")[1])
            cubes = cubes.strip().split(';')
            cubes = [note_set(ball_set) for ball_set in cubes]
            cubes = take_max_per_key(cubes)
            games[game] = cubes
    return games

test_games = process_input(example_string)
test_games

{1: {'blue': 6, 'red': 4, 'green': 2},
 2: {'blue': 4, 'green': 3, 'red': 1},
 3: {'green': 13, 'blue': 6, 'red': 20},
 4: {'green': 3, 'red': 14, 'blue': 15},
 5: {'red': 6, 'blue': 2, 'green': 3}}

And now, I want to check the test combinations.

In [5]:
test_combo = {'red': 12, 'green': 13, 'blue': 14}
def check_combo(combo, games):
    """
    Checks for which games the combo is a valid solution
    """
    results = [game for game in games if all(combo[key] >= games[game][key] for key in combo)]
    return results

Lastly, merging those functions together.

In [6]:
def part_one(input, combo_text, test = False):
    if test:
        games = process_input(input)
    else:
        with open(input, 'r+') as f:
            games = process_input(f.read())
    combo = note_set(combo_text)
    return sum(check_combo(combo, games))

assert(part_one(example_string, '12 red, 13 green, 14 blue', test=True) == 8)

In [7]:
part_one("./inputs/day02.txt", "12 red, 13 green, 14 blue")

2439

That's the right answer! You are one gold star ⭐ closer to restoring snow operations.

## Part Two 

Here, we want to get the power of a minimum set of each of the games. Thankfully, the function with processing the input I wrote for part one is all we need. Then I need to multiply the values, and we are all set.

In [8]:
test_dict = {'red': 1, 'green': 3, 'blue': 4}
from functools import reduce
# multiply all values in a dict
def calculate_power(dict_):
    return reduce(lambda x, y: x*y, dict_.values())

calculate_power(test_dict)

12

In [9]:
def part_two(input, test = False):
    if test:
        games = process_input(input)
    else:
        with open(input, 'r+') as f:
            games = process_input(f.read())
    power = [calculate_power(games[game]) for game in games]
    return sum(power)

In [10]:
assert(part_two(example_string, test=True) == 2286)

In [11]:
part_two("./inputs/day02.txt")

63711

That's the right answer! You are one gold star ⭐ closer to restoring snow operations.